In [20]:
import numpy as np
import pandas as pd
df = pd.read_csv('tests_perfomance/datasets/wine.csv')

In [10]:
df.quality.value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [2]:
def train_models(features, target, base_model, params):
    models = []
    for i, cat in enumerate(target.cat.categories[:-1]):
    #     print(i, cat, categ[:i + 1], categ[i + 1:])
        labels = [0] * len(categ[:i+1]) + [1] * len(categ[i + 1:])
        map = {cat: labels[j] for j, cat in enumerate(categ)}
        new_target = target.replace(map)
        mod = base_model(**params)
        mod.fit(features, new_target)
        models.append(mod)
    return models

In [12]:
from pandas.api.types import CategoricalDtype
target =  df.quality
categ = [3, 4, 5, 6, 7, 8]
cat_type = CategoricalDtype(categories=categ, ordered=True)
target = target.astype(cat_type)

In [5]:
from sklearn.linear_model import LogisticRegression

In [18]:
X = df.iloc[:, :-1]

In [15]:
models = train_models(X, target, LogisticRegression, dict(solver='lbfgs'))

/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs faile

In [17]:
from functools import reduce
prod = lambda x, y: x * y
def seq_predict_sample(sample, models):
    """
    Given list of binary classifiers and a sample, predict the most probable category index and the distribution
    """
    binary_probabilities = [mod.predict_proba(sample) for mod in models]
    neg_probs = [p[0][0] for p in binary_probabilities]
    pos_probs = [p[0][1] for p in binary_probabilities]
    bin_probabilities = []
    for i in range(len(models)):
        bin_prob = [p for p in pos_probs[:i]] + [neg_probs[i]]
        reduced = reduce(prod, bin_prob)
        bin_probabilities.append(reduced)
    bin_probabilities.append(reduce(prod, pos_probs))
    return np.argmax(bin_probabilities), bin_probabilities

In [27]:
y_pred = []
for i, x in X.iterrows():
    y_pred.append(categ[seq_predict_sample(x.values.reshape(1, -1), models)[0]])

In [29]:
from sklearn.metrics import accuracy_score

In [31]:
accuracy_score(target, y_pred)

0.5972482801751094

In [ ]:
reg_model = Log